In [1]:
#=============================================================================
# sentence-bert를 가지고 유사도 측정시, 최적화 시키는 예제임.
# 출처 : https://towardsdatascience.com/multilingual-text-similarity-matching-using-embedding-f79037459bf2
#
# Optimizer 방법
# 1. normalize_embeddings=True 
# 2. 내적 계산 : util.semantic_search 
#=============================================================================

from sentence_transformers import SentenceTransformer, util
import numpy as np
from os import sys
sys.path.append('..')
from myutils import GPU_info, seed_everything, mlogging

device = GPU_info()
seed_everything(111)
logger = mlogging(loggername='sbertembedding', logfilename='../../log/sbert-embedding')

model_path = "bongsoo/sentencebert_v1.2"

Optimizer = False  # 최적화 옵션 설정인 경우 True로 함.

True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30
logfilepath:../../log/sbert-embedding_2022-08-04.log


In [2]:
embedder = SentenceTransformer(model_path, cache_folder='./cache')
#embedder = SentenceTransformer('bongsoo/sentencebert_v1.2', device='cpu', cache_folder='./1')
print(embedder)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)


In [3]:
'''
# Corpus with example sentences
corpus = ['오늘은 날씨가 흐리고 비가 온다',
          '식당에서 밥을 먹었다',
          '관광 버스를 타고 여행을 한다',
          '낚시를 해서 물고기를 많이 잡았다',
          '동물원에서 호랑이를 보았다',
          '선거일에는 투표하러 가야 한다',
          '마트에 가서 맛있는 배를 샀다',
          '도서관에서 시험 공부 하고 있다',
          '야구장에 가서 열심히 응원 했다']

# Query sentences:
queries = ['구름 많고 매우 춥다',
           '비행기를 타고 간다',
           '어제 산에서 사슴를 봤다']
'''
'''
corpus = ['정치',
          '경제',
          '여행',
          '선거',
          '날씨',
          '서울',
          '축구',
          'IT',
          '금융']

# Query sentences:
queries = ['가장 가보고 싶은 여행지는?',
           '내 지역 투표장은 어디?',
           '요즘 가장 핫한 증권 소식은?']
'''
'''
corpus = ['서울은 대한민국에 수도이며, 정치 경제 중심지이다',
          '내년 경제 성장은 4%대 성장을 이룰거라 예상된다',
          '프랑스 파리는 전세계 관광객들이 매년 찾는 관광도시이다',
          '올해에는 대통령 선거와 지방선거가 동시에 열린다',
          '오늘 날씨는 비가 내리고 매우 춥다',
          '손홍민이 영국 프리미어 축구 경기에서 11번째 골을 넣었다',
          '건조한 날씨에 산불을 조심해야 한다',
          '윈도우11 OS에 검색 기능을 강화 하였다',
          '한국은행은 올해 하반기 금리를 동결했다',
          'Going on a trip',
          'it is raining',
          'stock market opened',
          'There is a voting for the class president election'
         ]
          

queries = ['여행',
           '투표',
           '증권',
           'IT']
'''

corpus = [
        'i love you', 
        'i am very happy', 
        'The weather is nice',
        'You do not have to do that. It is okay',
        'We expect the renovation to be finished by tomorrow evening.',
        'Thank you. We won’t make you regret it.',
        'Then I will see you at 10 a.m. tomorrow.',
        'The print cartridge is exhausted. Should I order more?'
         ]

queries = [
        '난 널 사랑해', 
        '난 매우 행복해', 
        '날씨가 좋다',
        '안 그래도 되는데 뭐. 괜찮아.',
        '내일 저녁까지 보수 공사가 끝날 것으로 예상합니다.',
        '감사합니다. 후회 없는 결정이 될 겁니다.',
        '그러면 오전 10시에 보도록 하죠. 내일 뵙겠습니다.',
        '프린트 카트리지가 다 떨어졌습니다. 더 주문할까요?'
        ]

In [5]:
import time

start = time.time()

logger.info(f'>>모델명: {model_path}')
logger.info(f'>>최적화: {Optimizer}')

# corpus 임베딩
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True, normalize_embeddings=Optimizer)
print(f'corpus_embed_type:{type(corpus_embeddings)}')
print(f'corpus_embed_shape:{corpus_embeddings.shape}')

# query 임베딩 하면서 corpus와 비교하여 유사도 출력
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity

top_k = min(5, len(corpus))

for query in queries:
    
    # 쿼리 임베딩 계산 
    query_embedding = embedder.encode(query, convert_to_tensor=True, normalize_embeddings=Optimizer)
   
    # 최적화 처리 
    if Optimizer == True:
        # util.semantic_search 로 내적 계산함.
        hits = util.semantic_search(query_embedding, corpus_embeddings, score_function=util.dot_score)
        hits = hits[0]
    
        logger.info(f"\n\n======================")
        logger.info(f"Query:{query}")

        for hit in hits[:top_k]:
            logger.info(f"{corpus[hit['corpus_id']]}({round(hit['score'], 4)})")
            
     # 최적화 처리가 아닌경우       
    else:
        cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
        cos_scores = cos_scores.cpu()

        #We use np.argpartition, to only partially sort the top_k results
        top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

        logger.info(f"\n\n======================")
        logger.info(f"Query:{query}")
        #print("\nTop 5 most similar sentences in corpus:")

        for idx in top_results[0:top_k]:
            logger.info(f"{corpus[idx].strip()} (Score: %.4f)" % (cos_scores[idx]))

logger.info(f"\n\n")
logger.info(f'>>처리시간: {time.time()-start:.4f}')
logger.info(f"\n\n======================")

2022-08-04 10:23:11,686 - sbertembedding - INFO - >>모델명: bongsoo/sentencebert_v1.2
2022-08-04 10:23:11,688 - sbertembedding - INFO - >>최적화: False
2022-08-04 10:23:11,720 - sbertembedding - INFO - 

2022-08-04 10:23:11,721 - sbertembedding - INFO - Query:난 널 사랑해
2022-08-04 10:23:11,722 - sbertembedding - INFO - i love you (Score: 0.6628)
2022-08-04 10:23:11,723 - sbertembedding - INFO - i am very happy (Score: 0.5538)
2022-08-04 10:23:11,724 - sbertembedding - INFO - Thank you. We won’t make you regret it. (Score: 0.2772)
2022-08-04 10:23:11,724 - sbertembedding - INFO - The weather is nice (Score: 0.1496)
2022-08-04 10:23:11,725 - sbertembedding - INFO - You do not have to do that. It is okay (Score: 0.1391)
2022-08-04 10:23:11,738 - sbertembedding - INFO - 

2022-08-04 10:23:11,738 - sbertembedding - INFO - Query:난 매우 행복해
2022-08-04 10:23:11,739 - sbertembedding - INFO - i am very happy (Score: 0.8904)
2022-08-04 10:23:11,740 - sbertembedding - INFO - You do not have to do that. It is

corpus_embed_type:<class 'torch.Tensor'>
corpus_embed_shape:torch.Size([8, 768])
